In [40]:
from sklearn.preprocessing import PolynomialFeatures 
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

from sklearn.metrics import accuracy_score 
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import seaborn as sns
import statsmodels.api as sm

import matplotlib.pyplot as plt
import numpy as np
import random
import pandas as pd

In [41]:
names = ['Class Name',
         'handicapped-infants',
         'water-project-cost-sharing',
         'adoption-of-the-budget-resolution',
         'physician-fee-freeze',
         'el-salvador-aid',
         'religious-groups-in-schools',
         'anti-satellite-test-ban',
         'aid-to-nicaraguan-contras',
         'mx-missile',
         'immigration',
         'synfuels-corporation-cutback',
         'education-spending',
         'superfund-right-to-sue',
         'crime',
         'duty-free-exports',
         'export-administration-act-south-africa']
data = pd.read_csv('house-votes-84.data', names=names)

data.head()


,Class Name,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [62]:
header = [""]
DATA = data.drop(columns='Class Name')
drops = []
for i in range(len(DATA)):
    for value in DATA.values[i, :]:
        if value == '?':
            drops.append(i)
Data = data.drop(index=drops)
Dat= data.drop(index=drops)
X = Data.drop(columns='Class Name')
y = Data.values[:, 0]

In [63]:
for i in range(len(y)):
    if y[i] == 'democrat':
        y[i] = 1
    else:
        y[i] = 0

for i in range(len(X.values)):
    for j in range(len(X.values[i, :])):
        if X.values[i, j] == 'y':
            X.values[i, j] = 1
        else:
            X.values[i, j] = 0

In [67]:
#Méthode de filtrage : par corrélation
def cor_selector(X, y):
    cor_list = []
    # On calcule la correlation entre chaque x et y
    for i in X.columns.tolist():
        cor = np.corrcoef(list(X[i].values), list(y))
        cor_list.append(cor[0,1])
    
     # feature name
    cor_feature = X.iloc[:,np.argsort(np.abs(cor_list))[-3:]].columns.tolist()
        
    # feature selection? 0 for not select, 1 for select
    cor_support = [True if i in cor_feature else False for i in names]

    return cor, cor_support, cor_feature

In [68]:
cor, cor_support, cor_feature = cor_selector(X, y)
print(str(len(cor_feature)), 'selected features')
print(cor_feature)

3 selected features
['education-spending', 'el-salvador-aid', 'physician-fee-freeze']


In [72]:
#We will select the features using chi square
Xx = X.values[:,1:17]
z=Dat.values[:,0]

In [73]:
Dat.head()

,Class Name,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
5,democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y
8,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,y
19,democrat,y,y,y,n,n,n,y,y,y,n,y,n,n,n,y,y
23,democrat,y,y,y,n,n,n,y,y,y,n,n,n,n,n,y,y
25,democrat,y,n,y,n,n,n,y,y,y,y,n,n,n,n,y,y


In [75]:
#Diviser les données en données d'entrainement et données de tests
## Décider de la taille des données pour chaque set
X_train, X_test, Y_train, Y_test = train_test_split(Xx,z,test_size=0.1)
test = SelectKBest(score_func=chi2, k=3)

#Fit the function for ranking the features by score

fit = test.fit(X_train, Y_train)

#Summarize scores 

np.set_printoptions(precision=3) 

print(fit.scores_)



[2.808e-02 4.551e+01 9.241e+01 5.243e+01 1.519e+01 2.338e+01 4.950e+01
 4.820e+01 1.189e-01 1.724e+01 5.802e+01 3.053e+01 3.169e+01 3.217e+01
 4.536e+00]


In [76]:
#Apply the transformation on to dataset

features = fit.transform(X_train)

#Summarize selected features 

print(features)


[[0 0 0]
 [1 1 1]
 [1 1 1]
 [0 0 0]
 [1 1 1]
 [0 0 0]
 [0 0 0]
 [1 1 1]
 [0 0 0]
 [0 1 0]
 [0 0 0]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 1 0]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [0 0 0]
 [1 1 0]
 [0 0 0]
 [0 1 0]
 [0 0 0]
 [0 1 0]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 0]
 [1 1 1]
 [1 1 0]
 [0 0 0]
 [1 1 1]
 [0 0 0]
 [0 0 0]
 [1 1 1]
 [1 1 1]
 [0 0 0]
 [1 1 1]
 [1 1 1]
 [0 0 0]
 [1 0 0]
 [0 0 0]
 [1 1 1]
 [0 0 0]
 [0 1 0]
 [0 0 0]
 [0 0 1]
 [0 0 0]
 [0 0 0]
 [1 1 1]
 [0 1 0]
 [1 1 1]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [1 1 1]
 [0 0 0]
 [1 1 0]
 [0 1 1]
 [1 1 1]
 [1 1 0]
 [1 1 1]
 [0 0 0]
 [1 1 0]
 [1 1 1]
 [0 0 0]
 [0 0 0]
 [0 1 1]
 [0 0 0]
 [1 1 1]
 [1 0 0]
 [1 1 1]
 [0 0 0]
 [0 1 1]
 [0 0 0]
 [1 1 1]
 [0 0 0]
 [1 1 1]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [1 1 1]
 [0 0 0]
 [1 1 1]
 [0 0 0]
 [0 1 1]
 [0 0 0]
 [0 0 0]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [0 1 1]
 [0 0 0]
 [0 0 1]
 [0 0 0]
 [0 1 0]
 [1 1 0]
 [0 0 0]
 [0 0 0]
 [0 1 0]
 [0 0 0]
 